# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [8]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [9]:
# Numeric column names
numeric_columns = X_train_numeric.columns

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [10]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [11]:
# Train the model 

normalized_input_model.fit(X_train, y_train, batch_size=32, epochs=150, validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [12]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [13]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, y_train_scaled, batch_size=32, epochs=150, validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.4654 - mse: 0.4654 - val_loss: 0.1773 - val_mse: 0.1773
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2517 - mse: 0.2517 - val_loss: 0.1991 - val_mse: 0.1991
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1926 - mse: 0.1926 - val_loss: 0.1407 - val_mse: 0.1407
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1787 - mse: 0.1787 - val_loss: 0.1406 - val_mse: 0.1406
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1522 - mse: 0.1522 - val_loss: 0.1661 - val_mse: 0.1661
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1429 - mse: 0.1429 - val_loss: 0.1372 - val_mse: 0.1372
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1381 - mse: 0.1381 - val_loss: 0.1267 - val_mse: 0.1267
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1234 - m

Epoch 63/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0230 - mse: 0.0230 - val_loss: 0.1264 - val_mse: 0.1264
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0222 - mse: 0.0222 - val_loss: 0.1290 - val_mse: 0.1290
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0215 - mse: 0.0215 - val_loss: 0.1322 - val_mse: 0.1322
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0216 - mse: 0.0216 - val_loss: 0.1309 - val_mse: 0.1309
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0212 - mse: 0.0212 - val_loss: 0.1275 - val_mse: 0.1275
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0212 - mse: 0.0212 - val_loss: 0.1289 - val_mse: 0.1289
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0206 - mse: 0.0206 - val_loss: 0.1301 - val_mse: 0.1301
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1287 - val_mse: 0.1287
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1291 - val_mse: 0.1291
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.1273 - val_mse: 0.1273
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.1286 - val_mse: 0.1286
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1302 - val_mse: 0.1302
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0101 - mse: 0.0101 - val_loss: 0.1281 - val_mse: 0.1281
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1288 - val_mse: 0.1288
Epoch 132/150
33/33 [==============================] - 0s 1ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [14]:
# Evaluate the model on training data

normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 943us/step - loss: 0.0079 - mse: 0.0079


[0.00788460299372673, 0.00788460299372673]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [15]:
# Evaluate the model on validate data

normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 898us/step - loss: 0.1283 - mse: 0.1283


[0.12828944623470306, 0.12828944623470306]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [16]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

28146.666671860512

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [17]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,), kernel_initializer='he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.5365 - mse: 0.5365 - val_loss: 0.2776 - val_mse: 0.2776
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2332 - mse: 0.2332 - val_loss: 0.2250 - val_mse: 0.2250
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1870 - mse: 0.1870 - val_loss: 0.2011 - val_mse: 0.2011
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1574 - mse: 0.1574 - val_loss: 0.1887 - val_mse: 0.1887
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1434 - mse: 0.1434 - val_loss: 0.1849 - val_mse: 0.1849
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1293 - mse: 0.1293 - val_loss: 0.1850 - val_mse: 0.1850
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1204 - mse: 0.1204 - val_loss: 0.1731 - val_mse: 0.1731
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1114 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0238 - mse: 0.0238 - val_loss: 0.2041 - val_mse: 0.2041
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0238 - mse: 0.0238 - val_loss: 0.2034 - val_mse: 0.2034
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0242 - mse: 0.0242 - val_loss: 0.2059 - val_mse: 0.2059
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0231 - mse: 0.0231 - val_loss: 0.2049 - val_mse: 0.2049
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0227 - mse: 0.0227 - val_loss: 0.2032 - val_mse: 0.2032
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0228 - mse: 0.0228 - val_loss: 0.2034 - val_mse: 0.2034
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0224 - mse: 0.0224 - val_loss: 0.2041 - val_mse: 0.2041
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0219 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0120 - mse: 0.0120 - val_loss: 0.2154 - val_mse: 0.2154
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0121 - mse: 0.0121 - val_loss: 0.2137 - val_mse: 0.2137
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0119 - mse: 0.0119 - val_loss: 0.2145 - val_mse: 0.2145
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.2140 - val_mse: 0.2140
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0115 - mse: 0.0115 - val_loss: 0.2155 - val_mse: 0.2155
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0115 - mse: 0.0115 - val_loss: 0.2163 - val_mse: 0.2163
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.2148 - val_mse: 0.2148
Epoch 132/150
33/33 [==============================] - 0s 1ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [18]:
# Evaluate the model on training data

he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 959us/step - loss: 0.0092 - mse: 0.0092


[0.009223182685673237, 0.009223182685673237]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on validate data

he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 896us/step - loss: 0.2157 - mse: 0.2157


[0.21574629843235016, 0.21574629843235016]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [20]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,), kernel_initializer='lecun_normal'))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.4705 - mse: 0.4705 - val_loss: 0.1797 - val_mse: 0.1797
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2301 - mse: 0.2301 - val_loss: 0.1486 - val_mse: 0.1486
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1931 - mse: 0.1931 - val_loss: 0.1308 - val_mse: 0.1308
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1710 - mse: 0.1710 - val_loss: 0.1170 - val_mse: 0.1170
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1553 - mse: 0.1553 - val_loss: 0.1237 - val_mse: 0.1237
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1437 - mse: 0.1437 - val_loss: 0.1186 - val_mse: 0.1186
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1290 - mse: 0.1290 - val_loss: 0.1092 - val_mse: 0.1092
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1210 - m

Epoch 63/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0228 - mse: 0.0228 - val_loss: 0.1200 - val_mse: 0.1200
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0227 - mse: 0.0227 - val_loss: 0.1205 - val_mse: 0.1205
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0222 - mse: 0.0222 - val_loss: 0.1227 - val_mse: 0.1227
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0221 - mse: 0.0221 - val_loss: 0.1187 - val_mse: 0.1187
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0215 - mse: 0.0215 - val_loss: 0.1194 - val_mse: 0.1194
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0207 - mse: 0.0207 - val_loss: 0.1204 - val_mse: 0.1204
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0211 - mse: 0.0211 - val_loss: 0.1198 - val_mse: 0.1198
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.

Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.1308 - val_mse: 0.1308
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.1314 - val_mse: 0.1314
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.1296 - val_mse: 0.1296
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0091 - mse: 0.0091 - val_loss: 0.1301 - val_mse: 0.1301
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0087 - mse: 0.0087 - val_loss: 0.1331 - val_mse: 0.1331
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0090 - mse: 0.0090 - val_loss: 0.1308 - val_mse: 0.1308
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0090 - mse: 0.0090 - val_loss: 0.1307 - val_mse: 0.1307
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on training data

lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 857us/step - loss: 0.0063 - mse: 0.0063


[0.006338715553283691, 0.006338715553283691]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on validate data

lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 999us/step - loss: 0.1338 - mse: 0.1338


[0.1337684690952301, 0.1337684690952301]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [24]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.3371 - mse: 0.3371 - val_loss: 0.1170 - val_mse: 0.1170
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1709 - mse: 0.1709 - val_loss: 0.1177 - val_mse: 0.1177
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1121 - mse: 0.1121 - val_loss: 0.1011 - val_mse: 0.1011
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1111 - mse: 0.1111 - val_loss: 0.1379 - val_mse: 0.1379
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0743 - mse: 0.0743 - val_loss: 0.0814 - val_mse: 0.0814
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0738 - mse: 0.0738 - val_loss: 0.0875 - val_mse: 0.0875
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0535 - mse: 0.0535 - val_loss: 0.1091 - val_mse: 0.1091
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0426 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0078 - mse: 0.0078 - val_loss: 0.0909 - val_mse: 0.0909
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0090 - mse: 0.0090 - val_loss: 0.0946 - val_mse: 0.0946
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0081 - mse: 0.0081 - val_loss: 0.0911 - val_mse: 0.0911
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0073 - mse: 0.0073 - val_loss: 0.0889 - val_mse: 0.0889
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0076 - mse: 0.0076 - val_loss: 0.0934 - val_mse: 0.0934
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0087 - mse: 0.0087 - val_loss: 0.0835 - val_mse: 0.0835
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0065 - mse: 0.0065 - val_loss: 0.1122 - val_mse: 0.1122
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0082 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0046 - mse: 0.0046 - val_loss: 0.0984 - val_mse: 0.0984
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0046 - mse: 0.0046 - val_loss: 0.0969 - val_mse: 0.0969
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0045 - mse: 0.0045 - val_loss: 0.0990 - val_mse: 0.0990
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0042 - mse: 0.0042 - val_loss: 0.0968 - val_mse: 0.0968
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0053 - mse: 0.0053 - val_loss: 0.1020 - val_mse: 0.1020
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0048 - mse: 0.0048 - val_loss: 0.1022 - val_mse: 0.1022
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0044 - mse: 0.0044 - val_loss: 0.1027 - val_mse: 0.1027
Epoch 132/150
33/33 [==============================] - 0s 1ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [25]:
# Evaluate the model on training data

rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 870us/step - loss: 0.0021 - mse: 0.0021  


[0.002050834009423852, 0.002050834009423852]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [26]:
# Evaluate the model on validate data

rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 889us/step - loss: 0.0904 - mse: 0.0904


[0.09039338678121567, 0.09039338678121567]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [27]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam', loss='mse', metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.3466 - mse: 0.3466 - val_loss: 0.1522 - val_mse: 0.1522
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1494 - mse: 0.1494 - val_loss: 0.1288 - val_mse: 0.1288
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0999 - mse: 0.0999 - val_loss: 0.1252 - val_mse: 0.1252
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0708 - mse: 0.0708 - val_loss: 0.1274 - val_mse: 0.1274
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0542 - mse: 0.0542 - val_loss: 0.1257 - val_mse: 0.1257
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0399 - mse: 0.0399 - val_loss: 0.1335 - val_mse: 0.1335
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0330 - mse: 0.0330 - val_loss: 0.1226 - val_mse: 0.1226
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0267 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0021 - mse: 0.0021 - val_loss: 0.1068 - val_mse: 0.1068
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0013 - mse: 0.0013 - val_loss: 0.1111 - val_mse: 0.1111
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 9.0225e-04 - mse: 9.0225e-04 - val_loss: 0.1077 - val_mse: 0.1077
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 6.4485e-04 - mse: 6.4485e-04 - val_loss: 0.1095 - val_mse: 0.1095
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 4.9004e-04 - mse: 4.9004e-04 - val_loss: 0.1085 - val_mse: 0.1085
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 5.0348e-04 - mse: 5.0348e-04 - val_loss: 0.1113 - val_mse: 0.1113
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 5.1382e-04 - mse: 5.1382e-04 - val_loss: 0.1097 - val_mse: 0.1097
Epoch 70/150
33/33 [=============================

33/33 [==============================] - 0s 2ms/step - loss: 5.5275e-04 - mse: 5.5275e-04 - val_loss: 0.1008 - val_mse: 0.1008
Epoch 124/150
33/33 [==============================] - 0s 2ms/step - loss: 6.0615e-04 - mse: 6.0615e-04 - val_loss: 0.1038 - val_mse: 0.1038
Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 6.0499e-04 - mse: 6.0499e-04 - val_loss: 0.1006 - val_mse: 0.1006
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 4.8097e-04 - mse: 4.8097e-04 - val_loss: 0.1030 - val_mse: 0.1030
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 4.4279e-04 - mse: 4.4279e-04 - val_loss: 0.1011 - val_mse: 0.1011
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 3.8916e-04 - mse: 3.8916e-04 - val_loss: 0.1056 - val_mse: 0.1056
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 4.7394e-04 - mse: 4.7394e-04 - val_loss: 0.0986 - val_mse: 0.0986
Epoch 130/150
33/33 [======

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [28]:
# Evaluate the model on training data

adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 778us/step - loss: 0.0114 - mse: 0.0114


[0.011406912468373775, 0.011406912468373775]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [29]:
# Evaluate the model on validate data

adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 882us/step - loss: 0.1075 - mse: 0.1075


[0.10750769823789597, 0.10750769823789597]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [30]:
# Evaluate the best model on test data

rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 1ms/step - loss: 0.1198 - mse: 0.1198


[0.11979620158672333, 0.11979620158672333]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [31]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

27199.006457449843

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.